# Entrega 2 - Árboles de decisión - Evaluación

### Grupo 13:
     - J. Aguirre  C.I: 4.773.509-6
     - A. Collazo C.I: 4.455.617-4
     - G. Núnez C.I: 4.785.081-2



## 1. Objetivo

El principal objetivo de esta tarea es implementar una versión extendida del algoritmo ID3 que sea capaz de procesar atributos numéricos y construir árboles de decisión a partir de esta característica; como también, poder evaluar los resultados obtenidos utilizando distintas medidas.

En la primera parte del análisis se utilizará el conjunto de datos *Iris* proporcionado por el equipo docente, que consta de tres clases de 50 instancias cada una y cuatro atributos numéricos. En la segunda parte se utilizará el conjunto de datos *Covertype*, también proporcionado por el equipo docente, que consta de 581012 instancias distribuidas entre siete clases y 54 atributos.


## 2. Diseño

En la siguiente sección se mencionan las decisiones de diseño utilizadas para la construcción de cada una de las partes del presente laboratorio.

### 2.1 Preprocesamiento de los datos

El preprocesamiento -como su nombre lo indica- consiste en preparar el conjunto de datos para luego ser procesado.

En una primera etapa se particiona el conjunto de datos en dos subconjuntos más pequeños, es decir, el primero consta del 80% de las instancias originales y se utiliza para entrenar (la construcción del árbol de decisión), mientras que el segundo subconjunto consta del restante 20% de las instancias y se utiliza para evaluar el modelo construido. Notar que al discriminar las instancias en dos subconjuntos disjuntos se evita que el árbol se ajuste por completo a la totalidad del conjunto original, y de esta forma, es posible evaluar el árbol construido mediante las instancias que no fue utilizadas en su construcción. Como la cantidad de instancias del conjunto *Iris* está equilibrada entre sus tres clases, la discriminación se realiza seleccionando al azar 80% de las instancias de cada clase, es decir, se seleccionan 120 de las 150 instancias, siendo 40 instancias de cada clase. En el caso del conjunto *Covertype*, la cantidad de instancias no está equilibrada por lo que la selección de datos requiere obtener un 80% de las instancias de cada clase, y de esta forma, seleccionar el 80% del total el conjunto manteniendo las proporciones. Análogamente el restante 20% de las tuplas se utilizan para evaluar el modelo.

Una vez discriminados ambos subconjuntos se procede a la etapa de preprocesamiento propiamente dicha. El preprocesamiento consiste en calcular la entropía y la ganancia del conjunto para cada atributo, y así, ordenarlos según su ganancia. Debido al reordenamiento de los atributos ponderados por su ganancia, es necesario también reordenar los valores de cada instancia de ambos subconjuntos para que coincida con el orden de los atributos. A continuación, se visualiza un ejemplo.

Sea el conjunto de atributos: {$A, B, C, D$} y sea el subconjunto de datos de entrenamiento: {{$a_1, b_1, c_1, d_1, n_1$}, {$a_2, b_2, c_2, d_2, n_2$}}; siendo respectivamente $a_n, b_n, c_n$ y $d_n$ el valor de la instancia $n_n$ correspondiente al atributo $A, B, C$ y $D$. Suponiendo que una vez calculada la ganancia de cada atributo el reordenamiento se realiza de la forma: {$B, C, A, D$}, entonces cada instancia del subconjunto se reordena de la forma: {{$b_1, c_1, a_1, d_1, n_1$}, {$b_2, c_2, a_2, d_2, n_2$}}, de modo tal que el orden de los valores de cada instancia coincide con el orden de los atributos, y por ende, también con los niveles del árbol.

Notar que la ganancia con la que se determina el orden de los atributos se calcula por única vez y utilizando todas las instancias del subconjunto de entrenamiento, lo que en principio podría resultar poco preciso. A continuación, se explica por qué.

Utilizando el mismo conjunto de atributos y el mismo subconjunto de datos de entrenamiento del ejemplo anterior, suponga que luego de calcular la ganancia de cada atributo se determina el siguiente orden: {$B, C, A, D$}. A continuación, se ordenan los valores de todas las instancias del subconjunto para que coincidan, por lo que se tiene: {{$b_1, c_1, a_1, d_1, n_1$}, {$b_2, c_2, a_2, d_2, n_2$}}. Hasta aquí es el mismo procedimiento del ejemplo anterior. Ahora bien, suponga que luego de filtrar las instancias por el atributo $B$, se decide calcular nuevamente la ganancia de cada atributo utilizando las restantes instancias del subconjunto. En este punto existen dos posibilidades. Por un lado, podría pasar que la cantidad de instancias permanezca invariante, en cuyo caso, la ganancia de cada atributo sería la misma. Por otro lado, podría pasar que la cantidad de instancias haya disminuido consecuencia del filtrado, provocando que la ganancia de los restantes atributos en el nuevo subconjunto de datos sea distinta. Por ejemplo, suponga que al recalcular la ganancia de los atributos el orden es: {$D, C, A$}, siendo el orden de los valores de las instancias: {{$d_1, c_1, a_1, n_1$}, {$d_2, c_2, a_2, n_2$}}. Si se unifica ambos cálculos en un orden general se tiene: {{$b_1, d_1, c_1, a_1, n_1$}, {$b_2, d_2, c_2, a_2, n_2$}} y no {{$b_1, c_1, a_1, d_1, n_1$}, {$b_2, c_2, a_2, d_2, n_2$}} como se calculó en un principio.

Para clasificar las instancias a partir de atributos numéricos es necesario especificar un criterio. Una opción válida -elegida para el presente laboratorio- es separar los valores en cuartiles. Esta decisión permite distribuir el conjunto de datos de manera uniforme y eliminar el ruido ocasionado por datos poco frecuentes (a diferencia de separar en promedios).

El último aspecto a tener en cuenta es que 44 de los 54 atributos del conjunto de datos Covertype son discretos (binarios), por lo que el enfoque para filtrar las instancias es ligeramente distinto. La primera opción -de hecho, la más performante en cuanto a costo computacional- es interpretar los 44 atributos como un número decimal en notación binaria, y así, utilizar los 44 atributos como un único atributo numérico. Dado que no se conoce el contexto (la semántica de cada uno), se optó por evitar la interpretación numérica con puntos de corte en pos de no agrupar valores anómalos o eventualmente imposibles. En contrapartida, los 44 atributos binarios se consideran como atributos independientes.



### 2.2 Procesamiento de los datos

El procesamiento de datos consiste en construir el árbol de decisión a partir del subconjunto de instancias de entrenamiento. Debido a los diferentes enfoques de las tres secciones se construyó un módulo independiente para cada sección. Para la sección a) se construyó el módulo ID3.py, para la sección b) se construyó el módulo Procesamiento.py, y para la sección c) se construyó el módulo Procesamiento.py. Cabe mencionar que cada módulo se encuentra adjunto dentro su respectiva carpeta.

Para representar el árbol de decisión se utilizó la clase llamada Node siendo cada instancia de esta, o una hoja, o un nodo intermedio (de ahora en adelante llamada nodo). El tipo queda determinado por la variable interna leaf, la cual toma el valor verdadero en caso de representar una hoja, o falso en caso contrario. A modo de relacionar la representación algorítmica con la realidad, diremos que los nodos intermedios del árbol representan los atributos mientras que las hojas representan los distintos valores del conjunto de datos.

En el caso del árbol multiclase cada hoja contendrá el valor de alguna de las clases del conjunto, por ejemplo, en el conjunto *Iris: Iris-setosa, Iris-versicolor*, entre otras, mientras que en el árbol uniclase cada hoja contendrá el valor *verdadero* o *falso* según corresponda su valor con la clase del árbol. A continuación, se visualiza una sección de un árbol multiclase para el conjunto *Iris*.

<img src="arbol1.png" alt="Drawing" style="width: 200px;"/>

*Árbol 1: Representación de un árbol multiclase para el conjunto de datos Iris. Notar que petal_width, petal_length, sepal_length y sepal_width corresponden a los atributos del conjunto y, por ende, a los nodos del árbol, mientras que Iris-setosa, Iris-versicolor e Iris-virginica corresponden a las hojas del árbol.*

A continuación, se visualiza una sección de un árbol binario para el conjunto *Iris*. Cabe mencionar que los subconjuntos de datos utilizados para construir ambos ejemplos son distintos y por ello no existe conexión alguna entre ellos.

<img src="arbol2.png" alt="Drawing" style="width: 200px;"/>

*Árbol 2: Representación de un árbol binario para el conjunto de datos Iris. Notar que petal_width, petal_length, sepal_length y sepal_width corresponden a los atributos del conjunto y, por ende, a los nodos del árbol, mientras que True y False corresponden a las hojas del árbol.*

Dado que *Árbol 2* fue construido para una clase particular del conjunto, el hecho de que una hoja sea *falso* significa que la combinación de atributos desde la raíz hasta esa hoja no se corresponde con esa clase. Análogamente para *verdadero*.
Además, se imprime la probabilidad de acierto para la instancia. Este resultado se utilizará para la votación entre los tres árboles.

### 2.3 Postprocesamiento de los datos

El postprocesamiento de datos consiste en analizar la calidad del árbol de decisión mediante el subconjunto de instancias de verificación. Para cada instancia del subconjunto se recorre el árbol desde la raíz hasta una hoja, eligiendo en cada nivel la rama (cuartil) que mejor se ajusta a su valor.

Al toparse con una hoja en un árbol multiclase simplemente verifica que su valor coincida con el de la hoja. En caso afirmativo significa que la decisión fue correcta, y en caso contrario significa que la decisión fue errónea.
En un árbol uniclase el procedimiento es similar a excepción de que existen más alternativas. A continuación, se detallan los posibles escenarios.

- Si la hoja tiene el valor verdadero, y el valor de la instancia de verificación coincide con el de la clase del árbol, entonces la decisión fue correcta (*verdadero positivo*).
- Si la hoja tiene el valor verdadero, y el valor de la instancia de verificación no coincide con el de la clase del árbol, entonces la decisión fue errónea (*falso positivo*).
- Si la hoja tiene el valor falso, y el valor de la instancia de verificación coincide con el de la clase del árbol, entonces la decisión fue errónea (*falso negativo*).
- Si la hoja tiene el valor falso, y el valor de la instancia de verificación no coincide con el de la clase del árbol, entonces la decisión fue correcta (*verdadero negativo*).

A continuación, se presenta el pseudocódigo que muestra la operativa anteriormente descripta:


```python
    def pre_procesamiento():
        generar_train()
        generar_test()
        calcular_entropia_total()
        calcular_puntos_corte() # si corresponde
        calcular_ganancia_atributos()
        ordenar_atributos()

    def procesamiento()
        id3()

    def post_procesamiento(arbol)
        resultados = [0,0,0,0] # v_pos | v_neg | f_pos | f_neg 
        para tupla en test:
            resultados.append(clasificar(tupla, arbol))
        return resultados
```

## 3. Experimentación

### 3.1 Parte a - Extensión del algoritmo ID3 con valores numéricos

Como se mencionó anteriormente, de las 150 instancias de conjunto Iris se utilizaron 120 para construir el árbol, y las restantes 30 para evaluar el modelo. Notar que la selección de las instancias de cada subconjunto se realiza de forma aleatoria. Además, cada atributo se particionó en cuatro cuartiles según se indica en el siguiente ejemplo. Cabe mencionar que los valores de *Figura 1* son meramente ilustrativos.

<table>
        <caption>Figura 1: Se visualiza un ejemplo de la división de los atributos en cuatro cuartiles, donde por ejemplo $i1 = 0,50$ pertenece al tercer cuartil mientras que $i2 = 0,91$ e $i3 = 0,75$ pertenecen al cuarto cuartil.</caption>
    <tr><td><img src="Cuartiles.png" alt="Drawing" style="width: 400px;"/></td></tr>
</table>    
 
A partir del árbol obtenido y el subconjunto de verificación se realizaron pruebas para conocer el desempeño en la clasificación de cada una de las clases. Cada prueba fue realizada con un subconjunto distinto. Los resultados se visualizan en *Tabla 1*.



<table>
  <caption>Tabla 1:Se visualiza el resultado de las diez pruebas realizadas.</caption>  
  <tr>
    <th>Identificador de prueba</th>
    <th>Instancias bien clasificadas</th>
    <th>Instancias mal clasificadas</th>
  </tr>
  <tr>
    <td>1</td>
    <td>26</td>
    <td>4</td>
  </tr>
  <tr>
    <td>2</td>
    <td>27</td>
    <td>3</td>
  </tr>
  <tr>
    <td>3</td>
    <td>26</td>
    <td>4</td>
  </tr>
  <tr>
    <td>4</td>
    <td>28</td>
    <td>2</td>
  </tr>
  <tr>
    <td>5</td>
    <td>28</td>
    <td>2</td>
  </tr>
  <tr>
    <td>6</td>
    <td>26</td>
    <td>4</td>
  </tr>
  <tr>
    <td>7</td>
    <td>27</td>
    <td>3</td>
  </tr>
  <tr>
    <td>8</td>
    <td>25</td>
    <td>5</td>
  </tr>
  <tr style="font-weight:bold">
    <td>9</td>
    <td>29</td>
    <td>1</td>
  </tr>
  <tr>
    <td>10</td>
    <td>27</td>
    <td>3</td>
  </tr>   
</table>

A continuación, se detalla la matriz de confusión para cada una de las pruebas realizadas donde se visualiza los resultados clasificados (correctamente o no) según cada clase.


<table>
  <tr>
    <th>#1</th>
    <th>Iris-setosa</th>
    <th>Iris-versicolor</th>
    <th>Iris-virginica</th>
  </tr>
  <tr>
    <td>Iris-setosa</td>
    <td>9</td>
    <td>1</td>
    <td>0</td>
  </tr>    
  <tr>
    <td>Iris-versicolor</td>
    <td>1</td>
    <td>8</td>
    <td>1</td>
  </tr>
  <tr>
    <td>Iris-virginica</td>
    <td>1</td>
    <td>0</td>
    <td>9</td>
  </tr>     
</table>


<table>
  <tr>
    <th>#2</th>
    <th>Iris-setosa</th>
    <th>Iris-versicolor</th>
    <th>Iris-virginica</th>
  </tr>
  <tr>
    <td>Iris-setosa</td>
    <td>10</td>
    <td>0</td>
    <td>0</td>
  </tr>    
  <tr>
    <td>Iris-versicolor</td>
    <td>1</td>
    <td>9</td>
    <td>0</td>
  </tr>
  <tr>
    <td>Iris-virginica</td>
    <td>0</td>
    <td>3</td>
    <td>7</td>
  </tr>     
</table>


<table>
  <tr>
    <th>#3</th>
    <th>Iris-setosa</th>
    <th>Iris-versicolor</th>
    <th>Iris-virginica</th>
  </tr>
  <tr>
    <td>Iris-setosa</td>
    <td>10</td>
    <td>0</td>
    <td>0</td>
  </tr>    
  <tr>
    <td>Iris-versicolor</td>
    <td>0</td>
    <td>10</td>
    <td>0</td>
  </tr>
  <tr>
    <td>Iris-virginica</td>
    <td>1</td>
    <td>3</td>
    <td>6</td>
  </tr>     
</table>


<table>
  <tr>
    <th>#4</th>
    <th>Iris-setosa</th>
    <th>Iris-versicolor</th>
    <th>Iris-virginica</th>
  </tr>
  <tr>
    <td>Iris-setosa</td>
    <td>10</td>
    <td>0</td>
    <td>0</td>
  </tr>    
  <tr>
    <td>Iris-versicolor</td>
    <td>0</td>
    <td>10</td>
    <td>0</td>
  </tr>
  <tr>
    <td>Iris-virginica</td>
    <td>2</td>
    <td>1</td>
    <td>7</td>
  </tr>     
</table>


<table>
  <tr>
    <th>#5</th>
    <th>Iris-setosa</th>
    <th>Iris-versicolor</th>
    <th>Iris-virginica</th>
  </tr>
  <tr>
    <td>Iris-setosa</td>
    <td>10</td>
    <td>0</td>
    <td>0</td>
  </tr>    
  <tr>
    <td>Iris-versicolor</td>
    <td>0</td>
    <td>10</td>
    <td>0</td>
  </tr>
  <tr>
    <td>Iris-virginica</td>
    <td>0</td>
    <td>3</td>
    <td>7</td>
  </tr>     
</table>


<table>
  <tr>
    <th>#6</th>
    <th>Iris-setosa</th>
    <th>Iris-versicolor</th>
    <th>Iris-virginica</th>
  </tr>
  <tr>
    <td>Iris-setosa</td>
    <td>10</td>
    <td>0</td>
    <td>0</td>
  </tr>    
  <tr>
    <td>Iris-versicolor</td>
    <td>0</td>
    <td>10</td>
    <td>0</td>
  </tr>
  <tr>
    <td>Iris-virginica</td>
    <td>1</td>
    <td>2</td>
    <td>7</td>
  </tr>     
</table>


<table>
  <tr>
    <th>#7</th>
    <th>Iris-setosa</th>
    <th>Iris-versicolor</th>
    <th>Iris-virginica</th>
  </tr>
  <tr>
    <td>Iris-setosa</td>
    <td>10</td>
    <td>0</td>
    <td>0</td>
  </tr>    
  <tr>
    <td>Iris-versicolor</td>
    <td>2</td>
    <td>7</td>
    <td>1</td>
  </tr>
  <tr>
    <td>Iris-virginica</td>
    <td>0</td>
    <td>0</td>
    <td>10</td>
  </tr>     
</table>


<table>
  <tr>
    <th>#8</th>
    <th>Iris-setosa</th>
    <th>Iris-versicolor</th>
    <th>Iris-virginica</th>
  </tr>
  <tr>
    <td>Iris-setosa</td>
    <td>8</td>
    <td>2</td>
    <td>0</td>
  </tr>    
  <tr>
    <td>Iris-versicolor</td>
    <td>0</td>
    <td>10</td>
    <td>0</td>
  </tr>
  <tr>
    <td>Iris-virginica</td>
    <td>1</td>
    <td>2</td>
    <td>7</td>
  </tr>     
</table>


<table>
  <tr>
    <th>#9</th>
    <th>Iris-setosa</th>
    <th>Iris-versicolor</th>
    <th>Iris-virginica</th>
  </tr>
  <tr>
    <td>Iris-setosa</td>
    <td>10</td>
    <td>0</td>
    <td>0</td>
  </tr>    
  <tr>
    <td>Iris-versicolor</td>
    <td>0</td>
    <td>9</td>
    <td>1</td>
  </tr>
  <tr>
    <td>Iris-virginica</td>
    <td>0</td>
    <td>0</td>
    <td>10</td>
  </tr>  
</table>


<table>
  <tr>
    <th>#10</th>
    <th>Iris-setosa</th>
    <th>Iris-versicolor</th>
    <th>Iris-virginica</th>
  </tr>
  <tr>
    <td>Iris-setosa</td>
    <td>10</td>
    <td>0</td>
    <td>0</td>
  </tr>    
  <tr>
    <td>Iris-versicolor</td>
    <td>0</td>
    <td>10</td>
    <td>0</td>
  </tr>
  <tr>
    <td>Iris-virginica</td>
    <td>0</td>
    <td>2</td>
    <td>8</td>
  </tr>     
</table>

Definimos la línea base para el acierto de la siguiente manera:

LB = max \{  \%50, max $(\frac{|c_i|}{|train set|}$) \}, $c_i \in Clases(target)$

Se puede apreciar que el acierto oscila entre 83,3% y 96,6% superando la línea base, por lo que el modelo obtenido es considerado correcto.
En cuanto al comportamiento del algoritmo entre las clases, se observa que las clasificaciones erróneas involucran a las clases *Iris-versicolor* e *Iris-virginica*. Esto podría explicarse en base a la similitud que tienen las mismas (según la descripción del conjunto de datos, *Iris-setosa* es linealmente separable de *Iris-virginica* e *Iris-versicolor*).


### 3.2 Parte b - Una clase vs. el resto

Luego de realizar algunas modificaciones al algoritmo de la sección 3.1 para crear un árbol de decisión por cada clase del conjunto, se procesó nuevamente el conjunto de datos *Iris*.
Se realizaron una serie de pruebas cada una con un subconjunto distinto seleccionado mediante el mismo procedimiento de la parte anterior.

Los resultados obtenidos se muestran a continuación. Las medidas seleccionadas para la evaluación son: *precisión, recuperación, fall-off, F1 y acierto*. La medida *F1* fue la elegida para medir el éxito, ya que no se tiene mayor información acerca de la ponderación entre recuperación y precisión para el contexto.

<table>
  <caption>Tabla 2: Iris-setosa vs el resto.</caption>  
  <tr>
    <th>Vp</th>
    <th>Vn</th>
    <th>Fp</th>
    <th>Fn</th>
    <th>Precisión</th>
    <th>Recuperación</th>
    <th>Fall-off</th>
    <th>F1</th>
    <th>Acierto</th>     
  </tr>
  <tr>
    <td>10</td>
    <td>20</td>
    <td>0</td>
    <td>0</td>
    <td style="font-weight:bold">1.00</td>
    <td style="font-weight:bold">1.00</td>
    <td style="font-weight:bold">0.00</td>
    <td style="font-weight:bold">1.00</td>
    <td style="font-weight:bold">1.00</td>
  </tr>
  <tr>
    <td>9</td>
    <td>20</td>
    <td>0</td>
    <td>1</td>
    <td style="font-weight:bold">1.00</td>
    <td>0.90</td>
    <td style="font-weight:bold">0.00</td>
    <td>0.94</td>
    <td>0.96</td>
  </tr>
  <tr>
    <td>9</td>
    <td>20</td>
    <td>0</td>
    <td>1</td>
    <td style="font-weight:bold">1.00</td>
    <td>0.90</td>
    <td style="font-weight:bold">0.00</td>
    <td>0.94</td>
    <td>0.96</td>
  </tr>
  <tr>
    <td>8</td>
    <td>14</td>
    <td>6</td>
    <td>2</td>
    <td>0.57</td>
    <td>0.80</td>
    <td>0.30</td>
    <td>0.66</td>
    <td>0.73</td>
  </tr>
  <tr>
    <td>10</td>
    <td>18</td>
    <td>2</td>
    <td>0</td>
    <td>0.83</td>
    <td style="font-weight:bold">1.00</td>
    <td>0.10</td>
    <td>0.90</td>
    <td>0.93</td>
  </tr>
  <tr>
    <td>10</td>
    <td>18</td>
    <td>2</td>
    <td>0</td>
    <td>0.83</td>
    <td style="font-weight:bold">1.00</td>
    <td>0.10</td>
    <td>0.90</td>
    <td>0.93</td>
  </tr>
  <tr>
    <td>10</td>
    <td>17</td>
    <td>3</td>
    <td>0</td>
    <td>0.76</td>
    <td style="font-weight:bold">1.00</td>
    <td>0.15</td>
    <td>0.86</td>
    <td>0.90</td>
  </tr>
  <tr>
    <td>10</td>
    <td>20</td>
    <td>0</td>
    <td>0</td>
    <td style="font-weight:bold">1.00</td>
    <td style="font-weight:bold">1.00</td>
    <td style="font-weight:bold">0.00</td>
    <td style="font-weight:bold">1.00</td>
    <td style="font-weight:bold">1.00</td>
  </tr>
  <tr>
    <td>10</td>
    <td>20</td>
    <td>0</td>
    <td>0</td>
    <td style="font-weight:bold">1.00</td>
    <td style="font-weight:bold">1.00</td>
    <td style="font-weight:bold">0.00</td>
    <td style="font-weight:bold">1.00</td>
    <td style="font-weight:bold">1.00</td>
  </tr>    
  <tr>
    <td>10</td>
    <td>19</td>
    <td>1</td>
    <td>0</td>
    <td>0.90</td>
    <td style="font-weight:bold">1.00</td>
    <td>0.05</td>
    <td>0.95</td>
    <td>0.96</td>
  </tr>   
</table>

<table>
  <caption>Tabla 3: Iris-versicolor vs el resto.</caption>  
  <tr>
    <th>Vp</th>
    <th>Vn</th>
    <th>Fp</th>
    <th>Fn</th>
    <th>Precisión</th>
    <th>Recuperación</th>
    <th>Fall-off</th>
    <th>F1</th>
    <th>Acierto</th>     
  </tr>
  <tr>
    <td>10</td>
    <td>19</td>
    <td>1</td>
    <td>0</td>
    <td>0.90</td>
    <td style="font-weight:bold">1.00</td>
    <td>0.05</td>
    <td>0.95</td>
    <td>0.96</td>
  </tr>
  <tr>
    <td>9</td>
    <td>19</td>
    <td>1</td>
    <td>1</td>
    <td>0.90</td>
    <td>0.90</td>
    <td>0.05</td>
    <td>0.90</td>
    <td>0.93</td>
  </tr>
  <tr>
    <td>9</td>
    <td>18</td>
    <td>2</td>
    <td>1</td>
    <td>0.81</td>
    <td>0.90</td>
    <td>0.10</td>
    <td>0.85</td>
    <td>0.90</td>
  </tr>
  <tr>
    <td>6</td>
    <td>20</td>
    <td>0</td>
    <td>4</td>
    <td style="font-weight:bold">1.00</td>
    <td>0.60</td>
    <td style="font-weight:bold">0.00</td>
    <td>0.75</td>
    <td>0.86</td>
  </tr>
  <tr>
    <td>8</td>
    <td>19</td>
    <td>1</td>
    <td>2</td>
    <td>0.88</td>
    <td>0.80</td>
    <td>0.05</td>
    <td>0.84</td>
    <td>0.90</td>
  </tr>
  <tr>
    <td>10</td>
    <td>20</td>
    <td>0</td>
    <td>0</td>
    <td style="font-weight:bold">1.00</td>
    <td style="font-weight:bold">1.00</td>
    <td style="font-weight:bold">0.00</td>
    <td style="font-weight:bold">1.00</td>
    <td style="font-weight:bold">1.00</td>
  </tr>
  <tr>
    <td>10</td>
    <td>19</td>
    <td>1</td>
    <td>0</td>
    <td>0.90</td>
    <td style="font-weight:bold">1.00</td>
    <td>0.05</td>
    <td>0.95</td>
    <td>0.96</td>
  </tr>
  <tr>
    <td>9</td>
    <td>20</td>
    <td>0</td>
    <td>1</td>
    <td style="font-weight:bold">1.00</td>
    <td>0.90</td>
    <td style="font-weight:bold">0.00</td>
    <td>0.94</td>
    <td>0.96</td>
  </tr>
  <tr>
    <td>10</td>
    <td>18</td>
    <td>2</td>
    <td>0</td>
    <td>0.83</td>
    <td style="font-weight:bold">1.00</td>
    <td>0.10</td>
    <td>0.90</td>
    <td>0.93</td>
  </tr>
  <tr>
    <td>9</td>
    <td>19</td>
    <td>1</td>
    <td>1</td>
    <td>0.90</td>
    <td>0.90</td>
    <td>0.05</td>
    <td>0.90</td>
    <td>0.93</td>
  </tr>    
</table>

<table>
  <caption>Tabla 4: Iris-virginica vs el resto.</caption>  
  <tr>
    <th>Vp</th>
    <th>Vn</th>
    <th>Fp</th>
    <th>Fn</th>
    <th>Precisión</th>
    <th>Recuperación</th>
    <th>Fall-off</th>
    <th>F1</th>
    <th>Acierto</th>     
  </tr>
  <tr>
    <td>9</td>
    <td>20</td>
    <td>0</td>
    <td>1</td>
    <td style="font-weight:bold">1.00</td>
    <td>0.90</td>
    <td style="font-weight:bold">0.00</td>
    <td>0.94</td>
    <td style="font-weight:bold">0.96</td>
  </tr>
  <tr>
    <td>9</td>
    <td>19</td>
    <td>1</td>
    <td>1</td>
    <td>0.90</td>
    <td>0.90</td>
    <td>0.05</td>
    <td>0.90</td>
    <td>0.93</td>
  </tr>
  <tr>
    <td>9</td>
    <td>19</td>
    <td>1</td>
    <td>1</td>
    <td>0.90</td>
    <td>0.90</td>
    <td>0.05</td>
    <td>0.90</td>
    <td>0.93</td>
  </tr>
  <tr>
    <td>3</td>
    <td>19</td>
    <td>1</td>
    <td>7</td>
    <td>0.75</td>
    <td>0.30</td>
    <td>0.05</td>
    <td>0.42</td>
    <td>0.73</td>
  </tr>
  <tr>
    <td>6</td>
    <td>19</td>
    <td>1</td>
    <td>4</td>
    <td>0.85</td>
    <td>0.60</td>
    <td>0.05</td>
    <td>0.70</td>
    <td>0.83</td>
  </tr>
  <tr>
    <td>8</td>
    <td>20</td>
    <td>0</td>
    <td>2</td>
    <td style="font-weight:bold">1.00</td>
    <td>0.80</td>
    <td style="font-weight:bold">0.00</td>
    <td>0.88</td>
    <td>0.93</td>
  </tr>
  <tr>
    <td>4</td>
    <td>20</td>
    <td>0</td>
    <td>6</td>
    <td style="font-weight:bold">1.00</td>
    <td>0.40</td>
    <td style="font-weight:bold">0.00</td>
    <td>0.57</td>
    <td>0.80</td>
  </tr>
  <tr>
    <td>10</td>
    <td>19</td>
    <td>1</td>
    <td>0</td>
    <td>0.90</td>
    <td style="font-weight:bold">1.00</td>
    <td>0.05</td>
    <td style="font-weight:bold">0.95</td>
    <td style="font-weight:bold">0.96</td>
  </tr>
  <tr>
    <td>8</td>
    <td>20</td>
    <td>0</td>
    <td>2</td>
    <td style="font-weight:bold">1.00</td>
    <td>0.80</td>
    <td style="font-weight:bold">0.00</td>
    <td>0.88</td>
    <td>0.93</td>
  </tr>
  <tr>
    <td>8</td>
    <td>19</td>
    <td>1</td>
    <td>2</td>
    <td>0.88</td>
    <td>0.80</td>
    <td>0.05</td>
    <td>0.84</td>
    <td>0.90</td>
  </tr>    
</table>

Para clasificar una instancia de forma definitiva se realiza una votación entre los árboles resultantes. Cada uno de los modelos predice su resultado con una probabilidad de acierto asociada. Si existe al menos una positiva se selecciona la de mayor probabilidad de este conjunto. En caso de que ninguno de los tres árboles declare como propia la instancia a clasificar, el resultado de la votación se determina por el árbol de decisión que haya retornado con menor probabilidad, que la instancia no le corresponde.

Los resultados de la votación se muestran en *Tabla 5*. En la mencionada tabla se visualiza la cantidad de aciertos  y errores luego de la votación.

<table>
  <caption>Tabla 5: Se visualiza el resultado de las diez pruebas realizadas.</caption>  
  <tr>
    <th>Prueba</th>
    <th>Aciertos</th>
    <th>Errores</th>
  </tr>
  <tr>
    <td>1</td>
    <td>27</td>
    <td>3</td>
  </tr>
  <tr style="font-weight:bold">
    <td>2</td>
    <td>29</td>
    <td>1</td>
  </tr>  
  <tr>
    <td>3</td>
    <td>28</td>
    <td>2</td>
  </tr>
  <tr>
    <td>4</td>
    <td>22</td>
    <td>8</td>
  </tr>
  <tr>
    <td>5</td>
    <td>18</td>
    <td>12</td>
  </tr>
  <tr>
    <td>6</td>
    <td>26</td>
    <td>4</td>
  </tr>
  <tr>
    <td>7</td>
    <td>18</td>
    <td>12</td>
  </tr>
  <tr>
    <td>8</td>
    <td>20</td>
    <td>10</td>
  </tr>
  <tr>
    <td>9</td>
    <td>19</td>
    <td>11</td>
  </tr>
  <tr>
    <td>10</td>
    <td>26</td>
    <td>4</td>
  </tr>
</table>

En todos los casos el porcentaje de acierto supera el 59% pero se presentan grandes variaciones.

### 3.3 Parte c - Aplicación para el conjunto CoverType

Para el conjunto *Covertype* se realiz

<table>
  <caption>Tabla 6: Una clase vs. el resto</caption>  
  <tr>
    <th>Id. clase</th>
    <th>Vp</th>
    <th>Vn</th>
    <th>Fp</th>
    <th>Fn</th>
    <th>Precisión</th>
    <th>Recuperación</th>
    <th>Fall-off</th>
    <th>F1</th>
    <th>Acierto</th>     
  </tr>
  <tr>
    <td align="left">1 - Spruce/Fir</td>  
    <td>23987</td>
    <td>53770</td>
    <td>20064</td>
    <td>18381</td>
    <td>0.54</td>
    <td>0.56</td>
    <td>0.27</td>
    <td>0.55</td>
    <td>0.66</td>
  </tr>
  <tr>
    <td align="left">2 - Lodgepole Pine</td>
    <td>31785</td>
    <td>39371</td>
    <td>20171</td>
    <td>24875</td>
    <td style="font-weight:bold">0.61</td>
    <td>0.56</td>
    <td>0.33</td>
    <td>0.58</td>
    <td>0.61</td>
  </tr>
  <tr>
    <td align="left">3 - Ponderosa Pine</td>
    <td>4501</td>
    <td>106163</td>
    <td>2888</td>
    <td>2650</td>
    <td>0.60</td>
    <td>0.62</td>
    <td>0.02</td>
    <td>0.61</td>
    <td>0.95</td>
  </tr>
  <tr>
    <td align="left">4 - Cottonwood/Willow</td>  
    <td>267</td>
    <td>115653</td>
    <td>0</td>
    <td>282</td>
    <td>1</td>
    <td>0.48</td>
    <td style="font-weight:bold">0</td>
    <td style="font-weight:bold">0.65</td>
    <td style="font-weight:bold">0.99</td>
  </tr>
  <tr>
    <td align="left">5 - Aspen</td>  
    <td>672</td>
    <td>113306</td>
    <td>997</td>
    <td>1227</td>
    <td>0.40</td>
    <td>0.35</td>
    <td>0.08</td>
    <td>0.37</td>
    <td>0.98</td>
  </tr>
  <tr>
    <td align="left">6 - douglas-fir</td>  
    <td>1457</td>
    <td>111458</td>
    <td>1271</td>
    <td>2016</td>
    <td>0.53</td>
    <td>0.41</td>
    <td>0.01</td>
    <td>0.46</td>
    <td>0.97</td>
  </tr>
  <tr>
    <td align="left">7 - Krummholz</td>  
    <td>3499</td>
    <td>108344</td>
    <td>3756</td>
    <td>603</td>
    <td>0.48</td>
    <td style="font-weight:bold">0.85</td>
    <td>0.03</td>
    <td>0.61</td>
    <td>0.96</td>
  </tr>    
</table>

<table>
  <caption>Tabla 7: Medida macro (multiclase) </caption>  
  <tr>
    <th>Macro (Iris)</th>
    <th>Macro (CoverType)</th>     
  </tr>
  <tr>
    <td style="font-weight:bold">0.896</td>
    <td>0.569</td>
  </tr>
</table>

<table>
  <caption>Tabla 8: Medida micro (booleanos) </caption>  
  <tr>
    <th>Micro (Iris)</th>
    <th>Micro (CoverType)</th>     
  </tr>
  <tr>
    <td style="font-weight:bold">0.983</td>
    <td>0.570</td>
  </tr>
</table>



## 4. Conclusión

Los resultados obtenidos en la sección a) de árboles multiclase sobre el conjunto Iris retorna un acierto medio de 89.95%, mayor a la línea base fijada, y por ende el modelo es calificado como exitoso.

Luego, en la sección b) la tasa de acierto del bosque y continuando la calificación mencionada anteriormente, el modelo es etiquetado como exitoso, aunque considerablemente menor a las tasas de los árboles individuales, e incluso irregular.  Esto puede deberse al criterio para evaluar el resultado de la votación, ya que cuando más de un árbol clasifiquen positivamente a una misma instancia para su clase y su probabilidad asociada sea igual, esta será asignada arbitrariamente a la primera clase en el orden de votación.

Por otra parte, el árbol multiclase construido para el conjunto Covertype retorna un acierto de 59% lo que lo vuelve también exitoso, aunque considerablemente menor que los modelos generados para el conjunto de datos Iris. Aquí el desafío radica en que los datos están desbalanceados, y el criterio seleccionado para abordar los datos discretos puede no haber sido el óptimo.

Futuras mejoras a los modelos desarrollados (incrementar su tasa de acierto), pueden desarrollarse en las siguientes líneas:
- Calcular las ganancias a nivel de cada nodo, y no en forma de preprocesamiento (como fue desarrollado en la sección de preprocesamiento).
- Generar distinta cantidad de ramas para un nodo, a partir de distintos puntos de corte (quintiles, deciles, percentiles) y distintos criterios (promedios, según ganancia generada, otros).
- Hacer validación cruzada.
- Cambiar el criterio de decisión de los bosques (no quedarse con la primera instancia de probabilidad máxima).
- Para el conjunto de datos *Covertype* en particular: experimentar con interpretar los datos discretos de forma continua (aunque está decisión no parece beneficiosa como se mencionó anteriormente) y afrontar el desbalance de las clases a partir de técnicas específicas para ello:  *«oversampling»* y *«undersampling»*.



## Apéndice 1

El algoritmo fue codificado en el lenguaje Python (versión 3.7.2). Es necesario instalar los paquetes *NumPy* (para facilitar los cálculos matemáticos) y *Colorama* (para decorar la salida de la terminal) ya que no pertenecen a los paquetes base del lenguaje. Se debe ejecutar los siguientes comandos en la terminal del dispositivo para instalar los mencionados paquetes.

 **python  -m  pip  install  numpy**
 
 **python  -m  pip  install  colorama**


### A

La sección a) consta de cuatro archivos explicados según el orden alfabético.

- data_set.txt - contiene el conjunto de datos Iris en su totalidad.
- ID3.py - contiene el código referente al algoritmo ID3.
- main.py - contiene el módulo principal de ejecución.
- PreProcessing.py - contiene el código referente al preprocesamiento del conjunto de datos.

Para ejecutar el algoritmo de la presente sección basta con ejecutar en la terminal:

 **python  main.py**


### B

La sección b) consta de cinco archivos explicados según el orden alfabético.

- data_set.txt - contiene el conjunto de datos Iris en su totalidad.
- main.py - contiene el módulo principal de ejecución.
- PostProcessing.py - contiene el código referente al postprocesamiento del conjunto de datos.
- PreProcessing.py - contiene el código referente al preprosamiento del conjunto de datos.
- Processing.py - contiene el código referente al algoritmo ID3.

Para ejecutar el algoritmo de la presente sección basta con ejecutar en la terminal:

**python  main.py**

### C

La sección c) consta de cinco archivos explicados según el orden alfabético.

- data_set.txt - contiene el conjunto de datos Iris en su totalidad.
- main.py - contiene el módulo principal de ejecución.
- PostProcessing.py - contiene el código referente al postprocesamiento del conjunto de datos.
- PreProcessing.py - contiene el código referente al preprosamiento del conjunto de datos.
- Processing.py - contiene el código referente al algoritmo ID3.

Para ejecutar el algoritmo de la presente sección basta con ejecutar en la terminal:

 **python  main.py  tipo_arbol**

Siendo *tipo_arbol* la modalidad deseada. Para generar un árbol multiclase se debe ingresar -1, mientras que para generar los correspondientes árboles uniclase se debe ingresar cualquier entero distinto de -1.

En el caso de las secciones a) y b), el preprocesamiento genera dos archivos llamados *test_set* y *train_set*, los cuales representan el conjunto de instancias de verificación y entrenamiento respectivamente. En el caso de la sección c), la discriminación de tuplas se encuentra calculado de antemano en los archivos *testing_set* y *training_set* (adjuntos en la entrega).

## Referencias

[1] Iris Data Set - R. A. Fisher, 1936 - https://archive.ics.uci.edu/ml/datasets/iris.

[2] Covertype Data Set - https://archive.ics.uci.edu/ml/datasets/Covertype.

[3] Análisis de datos - Clasificadores lineales - https://www.tamps.cinvestav.mx/~wgomez/diapositivas/RP/Clase06.pdf.

[4] Deep Learning unbalanced training data https://towardsdatascience.com/deep-learning-unbalanced-training-data-solve-it-like-this-6c528e9efea6.

[5] Understanding Quantiles: definitions and uses https://www.thoughtco.com/what-is-a-quantile-3126239.

[6] Accuracy, Precision, Recall or F1 - https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9.
